In [1]:
import pandas as pd
from src.lib.events import PowerBreakdown
from src.lib.plotting import MODE_COLORS
import plotly.graph_objects as go

In [2]:
DATA_FILE = "./data/DK-DK1.csv"
df = pd.read_csv(DATA_FILE, parse_dates=True, index_col="datetime")
df["production"] = df["production"].apply(PowerBreakdown.from_string).apply(lambda x: x._asdict())
df = df.join(pd.json_normalize(df["production"]).set_index(df.index))
df = df.drop(columns=["production", "capacity"])
df.head()

,created_at,updated_at,zone_key,source,biomass,coal,gas,geothermal,hydro,nuclear,oil,solar,unknown,wind,battery_storage,hydro_storage
datetime,,,,,,,,,,,,,,,,
2024-06-01 00:00:00+00:00,2024-06-01 01:05:06.934835 +00:00,2024-06-01 22:24:57.984572 +00:00,DE,entsoe.eu,4941.0,6022.0,3296.0,22.0,2045.0,None,373.0,0.0,181.0,10914.0,None,30.0
2024-06-01 00:15:00+00:00,2024-06-01 01:25:12.142551 +00:00,2024-06-01 22:24:57.984572 +00:00,DE,entsoe.eu,4949.0,5934.0,3332.0,22.0,2074.0,None,378.0,0.0,181.0,11118.0,None,199.0
2024-06-01 00:30:00+00:00,2024-06-01 01:25:12.142551 +00:00,2024-06-01 22:24:57.984572 +00:00,DE,entsoe.eu,4948.0,5880.0,3349.0,22.0,2098.0,None,375.0,0.0,181.0,11273.0,None,262.0
2024-06-01 00:45:00+00:00,2024-06-01 01:45:04.850232 +00:00,2024-06-01 22:24:57.984572 +00:00,DE,entsoe.eu,4925.0,5726.0,3350.0,22.0,2068.0,None,374.0,0.0,181.0,11517.0,None,337.0
2024-06-01 01:00:00+00:00,2024-06-01 02:04:56.250961 +00:00,2024-06-01 22:24:57.984572 +00:00,DE,entsoe.eu,4867.0,5473.0,3241.0,22.0,2048.0,None,375.0,0.0,181.0,11417.0,None,229.0


In [9]:
fig = go.Figure()
for mode in PowerBreakdown.get_production_modes():
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[mode],
            mode="lines",
            name=mode,
            stackgroup="one",
            fillcolor=MODE_COLORS[mode],
            line={"width": 0},
        )
    )

fig.update_layout(yaxis_title="Power Production [MW]", legend_title="Mode")
fig.show()